In [ ]:
df_new = pd.read_csv('two_hand_data.csv', header=None)

df_new['Mudra'] = ''

# Assign Mudra labels based on time ranges
time_column = df_new.columns[0]  # First column name (time)

# Map time ranges to Mudra labels
df_new.loc[df_new[time_column] < 35, 'Mudra'] = 'Bhumisparsa'
df_new.loc[(df_new[time_column] >= 35) & (df_new[time_column] < 43), 'Mudra'] = 'Varada'
df_new.loc[(df_new[time_column] >= 44) & (df_new[time_column] < 53), 'Mudra'] = 'Dhyana'
df_new.loc[(df_new[time_column] >= 54) & (df_new[time_column] < 61), 'Mudra'] = 'Abhaya'
df_new.loc[(df_new[time_column] >= 62) & (df_new[time_column] < 74), 'Mudra'] = 'Dharmachakra'
df_new.loc[(df_new[time_column] >= 75) & (df_new[time_column] < 84), 'Mudra'] = 'Vitarka'
df_new.loc[(df_new[time_column] >= 85) & (df_new[time_column] < 95), 'Mudra'] = 'Tarjani'
df_new.loc[(df_new[time_column] >= 96) & (df_new[time_column] < 105), 'Mudra'] = 'Namaskara'
df_new.loc[(df_new[time_column] >= 106) & (df_new[time_column] < 119), 'Mudra'] = 'Jnana'
df_new.loc[(df_new[time_column] >= 120) & (df_new[time_column] < 131), 'Mudra'] = 'Karana'
df_new.loc[(df_new[time_column] >= 132) & (df_new[time_column] < 142), 'Mudra'] = 'Ksepana'
df_new.loc[(df_new[time_column] >= 143) & (df_new[time_column] <= 150), 'Mudra'] = 'Uttarabodhi'

df_new.to_csv('two_hand_data+mudra_classification', index=False)

In [ ]:
import pandas as pd

df_new = pd.read_csv('one_hand_data.csv')

df_new['Mudra'] = ''

# Assign Mudra labels based on time ranges
time_column = df_new.columns[0]  # First column name (time)

# Map time ranges to Mudra labels
df_new.loc[df_new[time_column] < 32, 'Mudra'] = 'Bhumisparsa'
df_new.loc[(df_new[time_column] >= 35) & (df_new[time_column] < 48), 'Mudra'] = 'Varada'
df_new.loc[(df_new[time_column] >= 54) & (df_new[time_column] < 76), 'Mudra'] = 'Dhyana'
df_new.loc[(df_new[time_column] >= 80) & (df_new[time_column] < 96), 'Mudra'] = 'Abhaya'
df_new.loc[(df_new[time_column] >= 102) & (df_new[time_column] < 110), 'Mudra'] = 'Dharmachakra'
df_new.loc[(df_new[time_column] >= 112) & (df_new[time_column] < 122), 'Mudra'] = 'Vitarka'
df_new.loc[(df_new[time_column] >= 125) & (df_new[time_column] < 136), 'Mudra'] = 'Dharmachakra'
df_new.loc[(df_new[time_column] >= 140) & (df_new[time_column] < 153), 'Mudra'] = 'Tarjani'
df_new.loc[(df_new[time_column] >= 156) & (df_new[time_column] < 162), 'Mudra'] = 'Tarjani'
df_new.loc[(df_new[time_column] >= 167) & (df_new[time_column] < 181), 'Mudra'] = 'Namaskara'
df_new.loc[(df_new[time_column] >= 183) & (df_new[time_column] < 195), 'Mudra'] = 'Namaskara'
df_new.loc[(df_new[time_column] >= 199) & (df_new[time_column] < 220), 'Mudra'] = 'Jnana'
df_new.loc[(df_new[time_column] >= 224) & (df_new[time_column] < 243), 'Mudra'] = 'Karana'
df_new.loc[(df_new[time_column] >= 251) & (df_new[time_column] < 270), 'Mudra'] = 'Ksepana'
df_new.loc[(df_new[time_column] >= 278) & (df_new[time_column] < 283), 'Mudra'] = 'Ksepana'
df_new.loc[(df_new[time_column] >= 288) & (df_new[time_column] <= 311), 'Mudra'] = 'Uttarabodhi'

df_new.to_csv('one_hand_data+mudra_classification.csv', index=False)

In [9]:
df_VR1_data = pd.read_csv('two_hand_data+mudra_classification.csv')
df_VR2_data = pd.read_csv('one_hand_data+mudra_classification.csv')

# Add a 'Source' column to identify the origin dataset
df_VR1_data['Source'] = 'two_hand'
df_VR2_data['Source'] = 'one_hand'

# Combine the two dataframes
df_combined = pd.concat([df_VR1_data, df_VR2_data], ignore_index=True)

df_combined.to_csv('combined_data_with_classification.csv', index=False)

In [8]:
df_combined.head()

,29.99803,0.2136981,0.8744942,0.2430486,-0.6515009,0.1249106,0.1791945,0.7265214,5.960464E-08,0,...,L_PositionZ,L_RotationX,L_RotationY,L_RotationZ,L_RotationW,L_CurlIndex,L_CurlMiddle,L_CurlRing,L_CurlPinky,L_CurlThumb
0,30.01181,0.214019,0.874157,0.243098,-0.650972,0.124876,0.179243,0.726990,5.960464e-08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30.02595,0.214230,0.873697,0.243022,-0.650548,0.124907,0.179628,0.727268,5.960464e-08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30.04151,0.214439,0.873129,0.243302,-0.650006,0.124859,0.180480,0.727550,5.960464e-08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30.05398,0.214679,0.872187,0.243633,-0.649269,0.124558,0.181884,0.727910,5.960464e-08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30.06755,0.215062,0.871151,0.243994,-0.648441,0.124174,0.183002,0.728434,5.960464e-08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Drop non-numeric columns
df_numeric = df_new.select_dtypes(include=['number']).drop(columns=['Time'], errors='ignore')

# Handle missing values
df_numeric_filled = df_numeric.fillna(df_numeric.mean())

# Sample data if too large
sample_size = min(8428, len(df_numeric_filled))
df_sampled = df_numeric_filled.sample(sample_size, random_state=42)

# PCA for dimensionality reduction
pca = PCA(n_components=3)
X_pca_sampled = pca.fit_transform(df_sampled)

# Faster t-SNE settings
tsne = TSNE(n_components=2, perplexity=20, learning_rate=100, random_state=42)
X_tsne = tsne.fit_transform(X_pca_sampled)

# Plot t-SNE results
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=df_new['Mudra'], palette='viridis', alpha=0.5)
plt.title("Optimized t-SNE Visualization of Latent Space")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()
